# Advanced Querying Mongo

Importing libraries and setting up connection

In [9]:
import pandas as pd

In [1]:
from pymongo import MongoClient

In [2]:
client = MongoClient("localhost:27017")

In [3]:
db = client.get_database("IronHack")

In [4]:
c = db.get_collection("companies")

### 0. Vamos a cotillear unn poco el json a ver qué estructura tiene

In [13]:
ejemplo = c.find_one()

In [14]:
type(ejemplo)

dict

In [15]:
len(ejemplo)

38

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [19]:
filtro = {"name":"Babelgum"}

In [20]:
projec =  {"name":1}

In [26]:
uno = list(c.find(filtro,projec))

In [27]:
#uno

[{'_id': ObjectId('52cdef7c4bab8bd675297da0'), 'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [30]:
projec =  {"name":1, "number_of_employees":1}

In [31]:
dos = list(c.find({"number_of_employees": {"$gt": 5000}}, projec).sort("number_of_employees",-1).limit(20))

In [33]:
#dos

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [36]:
proj = {"name":1,"founded_year":1}
cond1={"founded_year": {"$gte": 2000}}
cond2={"founded_year": {"$lte": 2005}}

In [41]:
tres = list(c.find({"$and":[cond1,cond2]},proj))

In [43]:
#tres

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [44]:
valu={"ipo.valuation_amount": {"$gt": 100000000}}

In [45]:
found={"founded_year": {"$lte": 2010}}

In [46]:
proj = {"name":1,"ipo":1}

In [47]:
cuatro = list(c.find({"$and":[valu,found]},proj))

In [49]:
#cuatro

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [50]:
c1={"number_of_employees": {"$lt": 1000}}

In [51]:
c2={"founded_year": {"$lt": 2005}}

In [52]:
proj = {"name":1,"number_of_employees":1,"founded_year":1}

In [55]:
cinco=list(c.find({"$and":[c1,c2]},proj).sort("number_of_employees",-1).limit(10))

In [57]:
#cinco

### 6. All the companies that don't include the `partners` field.

In [58]:
seis = list(c.find({"partners": {"$exists": False}}))

In [59]:
seis

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [64]:
proj = {"name":1,"category_code":1}

In [65]:
seven = list(c.find({"category_code": {"$type": "null"}},proj))

In [68]:
len(seven) #  oojoooo!!!

2751

In [69]:
#seven

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [70]:
c1={"number_of_employees": {"$gt": 100}}

In [71]:
c2={"number_of_employees": {"$lt": 1000}}

In [72]:
proj = {"name":1,"number_of_employees":1}

In [79]:
ocho = list(c.find({"$and":[c1,c2]},proj).limit(3))

In [80]:
#ocho

### 9. Order all the companies by their IPO price in a descending order.

In [81]:
proj = {"name":1,"ipo.valuation_amount":1}

In [83]:
nueve = list(c.find({"ipo.valuation_amount": {"$exists": True}},proj).sort("ipo.valuation_amount",-1))

In [85]:
#nueve

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [86]:
proj = {"name":1,"number_of_employees":1}

In [90]:
diez = list(c.find({},proj).sort("number_of_employees",-1).limit(10))

In [92]:
#diez

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [93]:
cond={"founded_month": {"$gt": 6}}

In [96]:
proj = {"name":1,"founded_month":1}

In [99]:
once = list(c.find(cond,proj).limit(10))  #  aunque nos pidan mil, limito la búsqueda a sólo 10 para que no sea infinito

In [100]:
#once

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [117]:
c1={"acquisition.price_amount": {"$gt": 10000000}}

In [118]:
c2={"founded_year": {"$lt": 2000}}

In [119]:
proj = {"name":1,"acquisition.price_amount":1,"founded_year":1}

In [120]:
doce = list(c.find({"$and":[c1,c2]},proj))

In [122]:
#doce

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [127]:
proj = {"name":1,"acquisition":1}

In [128]:
trece = list(c.find({"acquisition.acquired_year": {"$gt": 2010}},proj).sort("acquisition.price_amount",-1))

In [129]:
len(trece)  #  ojo son muchos

736

In [130]:
#trece

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [158]:
proj = {"name":1,"founded_year":1}

In [166]:
catorce = list(c.find({"founded_year": {"$type": "int"}},proj).sort("founded_year",1).limit(10))

los estoy ordenando de más antiguo a más nuevo, pero empieza por los "none".
 Le meto un %type para que el founded_year sea un número

In [168]:
#catorce

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [172]:
proj = {"name":1,"founded_day":1,"acquisition.price_amount":1}

In [173]:
quince = list(c.find({"founded_day": {"$lte": 7}},proj).sort("acquisition.price_amount",-1).limit(10))

In [175]:
#quince

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [176]:
proj = {"name":1,"category_code":1,"number_of_employees":1}

In [177]:
c1={"category_code": "web"}

In [178]:
c2={"number_of_employees": {"$gt": 4000}}

In [179]:
dieciseis = list(c.find({"$and":[c1,c2]},proj).sort("number_of_employees",1))

In [181]:
#dieciseis

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [182]:
proj = {"acquisition.price_amount":1,"acquisition.price_currency_code":1}

In [183]:
c1={"acquisition.price_amount": {"$gt": 10000000}}

In [184]:
c2={"acquisition.price_currency_code": "EUR"}

In [185]:
diecisiete = list(c.find({"$and":[c1,c2]},proj))

In [187]:
#diecisiete

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [189]:
proj = {"name":1,"acquisition":1}

In [193]:
dieciocho = list(c.find({"acquisition.acquired_month": {"$lt": 4}},proj).limit(10)) 

In [195]:
#dieciocho

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [197]:
proj = {"name":1,"founded_year":1,"acquisition.acquired_year":1}

In [198]:
c1={"founded_year": {"$gt": 2000}}

In [199]:
c2={"founded_year": {"$lte": 2010}}

In [200]:
c3={"acquisition.acquired_year": {"$gt": 2011}}

In [201]:
diecinueve = list(c.find({"$and":[c1,c2,c3]},proj))

In [203]:
#diecinueve

### 20. All the companies that have been 'deadpooled' after the third year.

In [235]:
proj = {"name":1,"founded_year":1,"deadpooled_year":1}

In [236]:
c1={"founded_year": {"$type": "int"}}

In [237]:
c2={"founded_year": {"$type": "int"}}

In [258]:
c3={"$where":{"deadpooled_year"-"founded_year">3:True}}

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [260]:
c3={"deadpooled_year":{"$where"("deadpooled_year"-"founded_year">3):True}}

<>:1: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
<>:1: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
<ipython-input-260-3318028e532c>:1: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
  c3={"deadpooled_year":{"$where"("deadpooled_year"-"founded_year">3):True}}


TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [255]:
veinte = list(c.find({"$and":[c1,c2,c3]},proj))

InvalidDocument: cannot encode object: {<function <lambda> at 0x7f192c4b39d0>}, of type: <class 'set'>

In [ ]:
veinte